### Statistical Analysis of the Data

#### Time Complexity Analysis 
Here we look at the time taken and the number of nodes visited for the different algorithms

In [138]:
# Modules 
import pandas as pd
import numpy as np
import ast

def compute_row_wise_statistics(file_path):
    # Load data
    data = pd.read_csv(file_path)
    
    # Columns to analyze
    analysis_columns = ["Black score", "White score", "Pieces played",
                        "Black evaluated states", "White evaluated states",
                        "Number of nodes generated by Black", "Number of nodes generated by White",
                        "Time"]
    
    non_analysis_columns = [column for column in data.columns if column not in analysis_columns]
    
    # Compute a min, max, mean, std, and median across the columns and only keep one row
    aggregated_stats = {}
    for column in analysis_columns:
        column_data = data[column]
        if "evaluated states" in column or "nodes generated by" in column:
            column_data = column_data.apply(ast.literal_eval)
        else:
            column_data = column_data.apply(lambda x: [x]) # Convert the value into a list for structure consistency
            
        # Compute the statistics for each index of the lists
        min_values, max_values, mean_values, std_values, median_values = [], [], [], [], []
        for index in range(max([len(row) for row in column_data])):
            values = []
            for row in column_data:
                values.append(row[index]) if index < len(row) else values.append(0)
            min_values.append(min(values))
            max_values.append(max(values))
            mean_values.append(np.mean(values))
            std_values.append(np.std(values))
            median_values.append(np.median(values))
        
        aggregated_stats[column + " min"] = min_values if "evaluated states" in column or "nodes generated by" in column else min_values[0]
        aggregated_stats[column + " max"] = max_values if "evaluated states" in column or "nodes generated by" in column else max_values[0]
        aggregated_stats[column + " mean"] = mean_values if "evaluated states" in column or "nodes generated by" in column else mean_values[0]
        aggregated_stats[column + " std"] = std_values if "evaluated states" in column or "nodes generated by" in column else std_values[0]
        aggregated_stats[column + " median"] = median_values if "evaluated states" in column or "nodes generated by" in column else median_values[0]
        
    # Add the non-analysis columns
    for column in non_analysis_columns:
        aggregated_stats[column] = data[column].values[0]
    
    # Convert aggregated statistics into a DataFrame
    aggregated_stats_df = pd.DataFrame(aggregated_stats)
    
    # Put the non-analysis columns first
    columns = list(aggregated_stats_df.columns)
    columns = non_analysis_columns + [column for column in columns if column not in non_analysis_columns]
    aggregated_stats_df = aggregated_stats_df[columns]
    
    return aggregated_stats_df

def rows_to_list(df) -> pd.DataFrame:
    # Go through each row and concat the values into a list
    columns = {}   
    for column in df.columns:
        row_values = df[column].values
        # Concat the values into a list only if the column is not a repetition of the same value for all rows
        if len(set(row_values)) > 1: # Keep only the first 40 values as there shouldn't be more that around 32 moves for each player
            columns[column] = [list(row_values)[:40]]
        else:
            columns[column] = [row_values[0]]
    return pd.DataFrame(columns)

In [139]:
# Load data
import glob
import pandas as pd
file_paths = glob.glob("data/complexity/*.csv")
aggregated_stats_dfs = pd.DataFrame()
for file in file_paths:
    stats_df = compute_row_wise_statistics(file)
    new_df = rows_to_list(stats_df)
    aggregated_stats_dfs = pd.concat([aggregated_stats_dfs, new_df], ignore_index=True)

In [140]:
aggregated_stats_dfs.head(24)

,StrategyBlack,StrategyWhite,Depth,TableBlack,TableWhite,AlgorithmBlack,AlgorithmWhite,Black score min,Black score max,Black score mean,...,Number of nodes generated by White min,Number of nodes generated by White max,Number of nodes generated by White mean,Number of nodes generated by White std,Number of nodes generated by White median,Time min,Time max,Time mean,Time std,Time median
0,2,1,2,2,0,3,0,23,58,40.980000,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.028786,0.097934,0.044224,0.010384,0.042823
1,2,1,2,2,0,4,0,0,54,40.470000,...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.099...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.005505,0.161832,0.032771,0.020634,0.028469
2,2,1,4,2,0,3,0,24,56,40.320000,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",2.299836,13.465451,6.213564,2.059750,5.969241
3,2,1,4,2,0,4,0,25,60,40.480000,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.384127,1.815611,0.904395,0.296190,0.873614
4,2,1,6,2,0,3,0,25,51,41.000000,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",279.343929,582.921995,416.654451,108.489391,402.175941
5,2,1,6,2,0,4,0,42,46,43.666667,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",32.973344,39.178037,35.204304,2.816913,33.461533
6,3,1,2,2,0,3,0,12,64,39.880000,...,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.98, 0.97...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13999999...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001656,0.070559,0.016740,0.009180,0.016099
7,3,1,2,2,0,4,0,13,57,36.350000,...,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.98, 0.9...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0994987437106...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001097,0.173347,0.014553,0.019117,0.012244
8,3,1,4,2,0,3,0,13,61,38.100000,...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 0.99, 0.98, 0.96, 0.95, 0...","[0.0, 0.0, 0.0, 0.0, 0.09949874371066199, 0.13...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.030092,7.113760,2.101145,1.152596,2.025939
9,3,1,4,2,0,4,0,13,60,40.030000,...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 0.99, 0.98, 0...","[0.0, 0.0, 0.0, 0.0, 0.09949874371066199, 0.09...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.009345,0.943421,0.506164,0.194962,0.483702


In [141]:
import matplotlib.pyplot as plt

strategy_to_name = {
    2: 'Positional',
    3: 'Absolute',
    4: 'Mobility',
    5: 'Mixed (thresholds=[30, 55])',
}

algorithm_to_name = {
    3: 'Negamax',
    4: 'Negamax with Alpha-Beta Pruning',
}

def plot_nb_nodes(data, strategy_column='StrategyBlack', depth_column='Depth', algorithm_column='AlgorithmBlack', 
                  stats_string="Number of nodes generated by Black", save_folder=None):
    # Get unique depths, strategies, and algorithms for plotting
    unique_depths = data[depth_column].unique()
    unique_strategies = data[strategy_column].unique()
    unique_algorithms = data[algorithm_column].unique()
    
    # Plot data for each depth
    for depth in sorted(unique_depths):
        # Create a figure for all algorithms for the current depth
        fig_all, axes_all = plt.subplots(nrows=1, ncols=len(unique_algorithms), figsize=(20, 5), sharey=True)
        if len(unique_algorithms) == 1:
            axes_all = [axes_all]  # Ensure axes_all is iterable even if there's only one subplot
        
        for ax_all, algorithm in zip(axes_all, sorted(unique_algorithms)):
            # Create a separate figure for each algorithm
            fig, ax = plt.subplots(figsize=(10, 5))
            for strategy in unique_strategies:
                strategy_data = data[(data[strategy_column] == strategy) & (data[depth_column] == depth) & (data[algorithm_column] == algorithm)]
                if not strategy_data.empty:
                    # Extract indices for x-axis
                    indices = range(len(strategy_data[stats_string+' mean'].iloc[0]))
                    
                    # Plot each type of statistic on both figures
                    for ax_in_use in [ax, ax_all]:
                        ax_in_use.fill_between(indices, 
                                               strategy_data[stats_string+' min'].iloc[0], 
                                               strategy_data[stats_string+' max'].iloc[0], 
                                               alpha=0.5, label=f'{strategy_to_name[strategy]} Min-Max')
                        ax_in_use.plot(indices, strategy_data[stats_string+' mean'].iloc[0], 
                                       label=f'{strategy_to_name[strategy]} Mean')
                        ax_in_use.errorbar(indices, strategy_data[stats_string+' mean'].iloc[0], 
                                           yerr=strategy_data[stats_string+' std'].iloc[0], 
                                           fmt='o', label=f'{strategy_to_name[strategy]} STD')
            
            # Setup titles, labels, and layout for individual algorithm plot
            ax.set_title(f"{stats_string} at depth {depth}, {algorithm_to_name[algorithm]}")
            ax.set_xlabel("Move number")
            ax.set_ylabel(stats_string)
            ax.legend()
            plt.tight_layout()
            if save_folder:
                plt.savefig(f"{save_folder}/{stats_string}_depth_{depth}_{algorithm_to_name[algorithm]}.png")
            plt.close(fig)
            
            # Setup titles, labels for the all-algorithm plot
            ax_all.set_title(f"{algorithm_to_name[algorithm]}")
            ax_all.set_xlabel("Move number")
            ax_all.set_ylabel(stats_string)
            ax_all.legend()

        plt.tight_layout()
        # Save the combined plot for all algorithms at this depth
        if save_folder:
            plt.savefig(f"{save_folder}/{stats_string}_depth_{depth}_combined.png")
        plt.close(fig_all)
        
# plot_nb_nodes(aggregated_stats_dfs, save_folder="data/complexity")

In [162]:
def compare_stats(data, strategy_column='StrategyBlack', depth_column='Depth', filter_column='AlgorithmBlack',
                  stats_string="Number of nodes generated by Black"):
    unique_depths = data[depth_column].unique()
    unique_strategies = data[strategy_column].unique()
    unique_algorithms = data[filter_column].unique() if filter_column is not None else data.columns
    
    # Loop through each depth and strategy
    for depth in sorted(unique_depths):
        print(f"\n--- Depth {depth} ---")
        for strategy in unique_strategies:
            print(f"\nStrategy: {strategy_to_name[strategy]}")
            stats = {}
            # Collect statistics for each algorithm
            for algorithm in unique_algorithms:
                strategy_data = data[(data[strategy_column] == strategy) & 
                                     (data[depth_column] == depth) &
                                     ((data[filter_column] == algorithm) if filter_column is not None else True)]

                if not strategy_data.empty:
                    if isinstance(strategy_data[stats_string + ' mean'].iloc[0] , list):
                        mins = [min(x) for x in strategy_data[stats_string + ' min']]
                        maxs = [max(x) for x in strategy_data[stats_string + ' max']]
                        means = [np.mean(x) for x in strategy_data[stats_string + ' mean']]
                        stds = [np.mean(x) for x in strategy_data[stats_string + ' std']]
                    else:
                        mins = [strategy_data[stats_string + ' min'].iloc[0]]
                        maxs = [strategy_data[stats_string + ' max'].iloc[0]]
                        means = [strategy_data[stats_string + ' mean'].iloc[0]]
                        stds = [strategy_data[stats_string + ' std'].iloc[0]]
                    
                    stats[algorithm] = {
                        'Min of mins': min(mins),
                        'Max of maxs': max(maxs),
                        'Mean of means': np.mean(means),
                        'Mean of stds': np.mean(stds)
                    }
            
            # Compare NegamaxAlphaBeta/Negamax
            if filter_column == 'AlgorithmBlack' and 3 in stats and 4 in stats:
                print("\nNegamaxAlphaBeta/Negamax Comparison:")
                print(f"Max of maxs: {stats[4]['Max of maxs'] / stats[3]['Max of maxs'] * 100:.2f}% ({stats[4]['Max of maxs']:.2f} vs {stats[3]['Max of maxs']:.2f})")
                print(f"Mean of means: {stats[4]['Mean of means'] / stats[3]['Mean of means'] * 100:.2f}% ({stats[4]['Mean of means']:.2f} vs {stats[3]['Mean of means']:.2f})")
                print(f"Mean of stds: {stats[4]['Mean of stds'] / stats[3]['Mean of stds'] * 100:.2f}% ({stats[4]['Mean of stds']:.2f} vs {stats[3]['Mean of stds']:.2f})")
            
            # Compare between tables
            if filter_column in ['TableBlack', 'TableWhite'] and 1 in stats and 2 in stats:
                print("\nHeuristic 2 / Heuristic 1 Comparison:")
                print(f"Max of maxs: {stats[2]['Max of maxs'] / stats[1]['Max of maxs'] * 100:.2f}% ({stats[2]['Max of maxs']:.2f} vs {stats[1]['Max of maxs']:.2f})")
                print(f"Mean of means: {stats[2]['Mean of means'] / stats[1]['Mean of means'] * 100:.2f}% ({stats[2]['Mean of means']:.2f} vs {stats[1]['Mean of means']:.2f})")
                print(f"Mean of stds: {stats[2]['Mean of stds'] / stats[1]['Mean of stds'] * 100:.2f}% ({stats[2]['Mean of stds']:.2f} vs {stats[1]['Mean of stds']:.2f})")
                
                
def overall(data, column, depth_column="Depth", strategy_column='StrategyBlack'):
    """Compute the overall statistics for a column per strategy"""
    unique_strategies = data[strategy_column].unique()
    for depth in data[depth_column].unique():
        print(f"\n--- Depth {depth} ---")
        for strategy in unique_strategies:
            strategy_data = data[(data[strategy_column] == strategy) & (data[depth_column] == depth)]
            if not strategy_data.empty:
                print(f"\n--- Strategy {strategy_to_name[strategy]} ---")
                print(f"Min: {min(strategy_data[column + ' min'])}")
                print(f"Max: {max(strategy_data[column + ' max'])}")
                print(f"Mean: {np.mean(strategy_data[column + ' mean'])}")
                print(f"STD: {np.mean(strategy_data[column + ' std'])}")

In [143]:
compare_stats(aggregated_stats_dfs.copy(), stats_string="Number of nodes generated by Black")


--- Depth 2 ---

Strategy: Positional

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 58.80% (137.00 vs 233.00)
Mean of means: 57.52% (31.93 vs 55.51)
Mean of stds: 65.20% (10.91 vs 16.73)

Strategy: Absolute

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 64.90% (159.00 vs 245.00)
Mean of means: 55.79% (25.79 vs 46.22)
Mean of stds: 63.92% (14.11 vs 22.08)

Strategy: Mobility

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 54.04% (147.00 vs 272.00)
Mean of means: 53.95% (32.56 vs 60.36)
Mean of stds: 60.26% (11.02 vs 18.30)

Strategy: Mixed (thresholds=[30, 55])

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 61.09% (146.00 vs 239.00)
Mean of means: 58.35% (31.78 vs 54.45)
Mean of stds: 62.72% (10.87 vs 17.34)

--- Depth 4 ---

Strategy: Positional

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 13.19% (7103.00 vs 53861.00)
Mean of means: 14.77% (787.08 vs 5329.77)
Mean of stds: 16.35% (467.24 vs 2856.85)

Strategy: Absolute

NegamaxAlphaBeta/Negamax Comparison:
Max of

In [144]:
compare_stats(aggregated_stats_dfs.copy(), stats_string="Time")


--- Depth 2 ---

Strategy: Positional

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 165.25% (0.16 vs 0.10)
Mean of means: 74.10% (0.03 vs 0.04)
Mean of stds: 198.71% (0.02 vs 0.01)

Strategy: Absolute

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 245.68% (0.17 vs 0.07)
Mean of means: 86.94% (0.01 vs 0.02)
Mean of stds: 208.26% (0.02 vs 0.01)

Strategy: Mobility

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 114.02% (0.24 vs 0.21)
Mean of means: 54.55% (0.06 vs 0.12)
Mean of stds: 83.37% (0.02 vs 0.03)

Strategy: Mixed (thresholds=[30, 55])

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 116.59% (0.22 vs 0.19)
Mean of means: 67.22% (0.06 vs 0.08)
Mean of stds: 140.54% (0.04 vs 0.03)

--- Depth 4 ---

Strategy: Positional

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 13.48% (1.82 vs 13.47)
Mean of means: 14.56% (0.90 vs 6.21)
Mean of stds: 14.38% (0.30 vs 2.06)

Strategy: Absolute

NegamaxAlphaBeta/Negamax Comparison:
Max of maxs: 13.26% (0.94 vs 7.11)
Mean of mean

#### Same Thing but with the Championship Data

In [145]:
# Load data
import glob
import pandas as pd
file_paths = glob.glob("data/championship/*.csv")
aggregated_stats_dfs = pd.DataFrame()
for file in file_paths:
    stats_df = compute_row_wise_statistics(file)
    new_df = rows_to_list(stats_df)
    aggregated_stats_dfs = pd.concat([aggregated_stats_dfs, new_df], ignore_index=True)

In [146]:
aggregated_stats_dfs.head(108)

,StrategyBlack,StrategyWhite,Depth,TableBlack,TableWhite,AlgorithmBlack,AlgorithmWhite,Black score min,Black score max,Black score mean,...,Number of nodes generated by White min,Number of nodes generated by White max,Number of nodes generated by White mean,Number of nodes generated by White std,Number of nodes generated by White median,Time min,Time max,Time mean,Time std,Time median
0,2,2,2,1,1,4,4,13,50,33.67,...,"[10, 17, 36, 27, 48, 27, 29, 27, 25, 28, 27, 3...","[14, 21, 46, 36, 60, 80, 101, 118, 108, 107, 1...","[12.21, 19.0, 41.7, 31.62, 56.09, 47.99, 62.71...","[1.8237050200073475, 2.0, 3.9127995093027708, ...","[12.5, 19.0, 44.0, 30.0, 57.5, 42.0, 61.0, 67....",0.042013,0.276368,0.059678,0.028809,0.053641
1,2,2,2,1,2,4,4,11,49,25.13,...,"[17, 14, 8, 12, 12, 17, 13, 12, 18, 14, 16, 15...","[17, 33, 60, 62, 85, 84, 118, 157, 99, 109, 14...","[17.0, 23.81, 27.75, 32.31, 40.03, 42.53, 49.7...","[0.0, 6.0953999048462775, 11.411726425041918, ...","[17.0, 24.0, 25.0, 32.0, 40.0, 36.0, 48.5, 50....",0.031584,0.223238,0.047088,0.025330,0.042511
2,2,2,2,2,1,4,4,20,56,40.34,...,"[10, 9, 16, 20, 17, 21, 17, 17, 7, 19, 12, 18,...","[14, 21, 46, 55, 58, 71, 83, 77, 94, 114, 82, ...","[12.22, 15.38, 30.91, 32.01, 34.66, 37.2, 41.0...","[1.8792551716038992, 3.662731221370195, 8.7807...","[14.0, 17.0, 30.0, 32.0, 31.0, 35.0, 35.5, 32....",0.030136,0.223428,0.046774,0.023737,0.042312
3,2,2,2,2,2,4,4,7,56,32.36,...,"[17, 16, 15, 16, 16, 19, 25, 19, 20, 22, 14, 1...","[17, 33, 61, 62, 98, 93, 108, 110, 113, 139, 1...","[17.0, 23.69, 37.0, 38.06, 48.21, 55.36, 60.41...","[0.0, 5.36040110439508, 9.846826900072937, 11....","[17.0, 20.0, 38.5, 37.0, 44.0, 56.0, 56.0, 57....",0.035454,0.237087,0.055933,0.025985,0.050498
4,2,2,4,1,1,4,4,11,51,32.19,...,"[88, 185, 242, 214, 234, 432, 388, 304, 252, 3...","[168, 335, 718, 1165, 1271, 2378, 2131, 2652, ...","[139.52, 293.44, 439.36, 629.55, 761.34, 1089....","[35.00527960179721, 60.4008807882799, 141.0367...","[168.0, 331.0, 455.0, 632.5, 755.0, 877.0, 107...",1.052835,4.484553,1.890052,0.526095,1.768745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,5,5,4,2,2,4,4,4,58,29.69,...,"[173, 363, 239, 235, 584, 557, 602, 806, 397, ...","[269, 530, 1556, 2380, 3265, 3383, 5292, 5179,...","[207.84, 413.93, 942.32, 1269.46, 1615.88, 202...","[33.75699038717759, 59.75353629702596, 352.657...","[195.0, 375.0, 939.0, 1229.0, 1694.0, 1778.0, ...",1.945327,6.921165,3.276641,0.966921,3.057079
104,5,5,6,1,1,4,4,12,46,33.60,...,"[1819, 2533, 6834, 4291, 5332, 13055, 20400, 2...","[2412, 3536, 10274, 14167, 24100, 19494, 38923...","[2056.2, 3072.4, 8228.8, 8333.8, 11647.6, 1564...","[290.5094834940849, 319.92036509106447, 1180.4...","[1819.0, 3072.0, 7671.0, 5630.0, 6055.0, 14051...",32.123373,109.749609,78.050787,36.039214,104.290822
105,5,5,6,1,2,4,4,34,52,39.80,...,"[2165, 6625, 899, 3424, 4627, 9429, 19686, 319...","[4297, 9214, 2273, 4401, 12682, 42362, 55609, ...","[3401.4, 8431.8, 1837.6, 3823.4, 9539.0, 16083...","[688.403834968981, 1038.4046224858594, 562.995...","[3515.0, 9214.0, 2273.0, 3764.0, 9678.0, 9529....",20.436180,95.066309,73.126312,27.023046,84.554180
106,5,5,6,2,1,4,4,18,54,35.60,...,"[1819, 4258, 3506, 6582, 14389, 24180, 6954, 1...","[2412, 4258, 8665, 9664, 20782, 26389, 21086, ...","[2056.2, 4258.0, 5369.4, 7729.8, 18539.2, 2551...","[290.5094834940849, 0.0, 2304.0376385814534, 1...","[1819.0, 4258.0, 3506.0, 6582.0, 19175.0, 2638...",38.342450,82.986742,63.590366,19.782634,75.603749


In [147]:
compare_stats(aggregated_stats_dfs.copy(), strategy_column='StrategyBlack', stats_string="Black score", filter_column='TableBlack')


--- Depth 2 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 112.00% (56.00 vs 50.00)
Mean of means: 119.81% (40.34 vs 33.67)
Mean of stds: 74.22% (8.31 vs 11.20)

Strategy: Absolute

Strategy: Mobility

Strategy: Mixed (thresholds=[30, 55])

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 116.00% (58.00 vs 50.00)
Mean of means: 111.36% (31.26 vs 28.07)
Mean of stds: 118.48% (10.94 vs 9.23)

--- Depth 4 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 111.76% (57.00 vs 51.00)
Mean of means: 123.77% (39.84 vs 32.19)
Mean of stds: 99.24% (9.33 vs 9.40)

Strategy: Absolute

Strategy: Mobility

Strategy: Mixed (thresholds=[30, 55])

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 112.73% (62.00 vs 55.00)
Mean of means: 102.79% (33.56 vs 32.65)
Mean of stds: 119.15% (11.94 vs 10.02)

--- Depth 6 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 109.52% (46.00 vs 42.00)
Mean of means: 146.67% (39.60 

In [148]:
compare_stats(aggregated_stats_dfs.copy(), strategy_column='StrategyWhite', stats_string="White score", filter_column='TableWhite')


--- Depth 2 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 103.92% (53.00 vs 51.00)
Mean of means: 128.22% (38.85 vs 30.30)
Mean of stds: 83.51% (9.34 vs 11.19)

Strategy: Absolute

Strategy: Mobility

Strategy: Mixed (thresholds=[30, 55])

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 115.69% (59.00 vs 51.00)
Mean of means: 110.02% (31.28 vs 28.43)
Mean of stds: 115.42% (10.44 vs 9.05)

--- Depth 4 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 105.66% (56.00 vs 53.00)
Mean of means: 97.70% (31.06 vs 31.79)
Mean of stds: 199.38% (18.76 vs 9.41)

Strategy: Absolute

Strategy: Mobility

Strategy: Mixed (thresholds=[30, 55])

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 113.21% (60.00 vs 53.00)
Mean of means: 90.55% (29.52 vs 32.60)
Mean of stds: 218.67% (18.74 vs 8.57)

--- Depth 6 ---

Strategy: Positional

Heuristic 2 / Heuristic 1 Comparison:
Max of maxs: 111.32% (59.00 vs 53.00)
Mean of means: 114.05% (42.20 v

In [163]:
overall(aggregated_stats_dfs.copy(),  column='Black score')


--- Depth 2 ---

--- Strategy Positional ---
Min: 0
Max: 58
Mean: 35.458333333333336
STD: 9.62467625979703

--- Strategy Absolute ---
Min: 0
Max: 62
Mean: 24.293333333333337
STD: 10.101788577312341

--- Strategy Mobility ---
Min: 0
Max: 62
Mean: 26.97
STD: 12.160360965176416

--- Strategy Mixed (thresholds=[30, 55]) ---
Min: 0
Max: 62
Mean: 34.086666666666666
STD: 10.987617147481513

--- Depth 4 ---

--- Strategy Positional ---
Min: 0
Max: 59
Mean: 32.7
STD: 9.03570095182245

--- Strategy Absolute ---
Min: 0
Max: 61
Mean: 22.435
STD: 10.03634055939672

--- Strategy Mobility ---
Min: 0
Max: 63
Mean: 30.713333333333335
STD: 11.977417357224217

--- Strategy Mixed (thresholds=[30, 55]) ---
Min: 0
Max: 64
Mean: 34.940000000000005
STD: 11.650047543501584

--- Depth 6 ---

--- Strategy Positional ---
Min: 0
Max: 52
Mean: 29.566666666666666
STD: 10.606901547243325

--- Strategy Absolute ---
Min: 2
Max: 59
Mean: 26.33015873015873
STD: 8.536471568522327

--- Strategy Mobility ---
Min: 0
Max: 61

In [164]:
overall(aggregated_stats_dfs.copy(),  column='White score', strategy_column='StrategyWhite')


--- Depth 2 ---

--- Strategy Positional ---
Min: 0
Max: 58
Mean: 34.953333333333326
STD: 9.871917857233216

--- Strategy Absolute ---
Min: 0
Max: 58
Mean: 24.765000000000004
STD: 9.954635916338612

--- Strategy Mobility ---
Min: 0
Max: 62
Mean: 29.325000000000003
STD: 11.069492041477508

--- Strategy Mixed (thresholds=[30, 55]) ---
Min: 0
Max: 62
Mean: 33.524166666666666
STD: 11.562697793136374

--- Depth 4 ---

--- Strategy Positional ---
Min: 0
Max: 57
Mean: 31.90916666666666
STD: 11.554167445882685

--- Strategy Absolute ---
Min: 0
Max: 63
Mean: 23.343333333333334
STD: 10.259030698293873

--- Strategy Mobility ---
Min: 0
Max: 64
Mean: 27.855
STD: 11.187952276191007

--- Strategy Mixed (thresholds=[30, 55]) ---
Min: 0
Max: 62
Mean: 34.2925
STD: 12.562929303544527

--- Depth 6 ---

--- Strategy Positional ---
Min: 7
Max: 59
Mean: 32.96666666666666
STD: 8.770063167012337

--- Strategy Absolute ---
Min: 0
Max: 58
Mean: 21.987301587301587
STD: 12.138572828617043

--- Strategy Mobility 